<a href="https://colab.research.google.com/github/jaesanglee95/ESC-20SUMMER/blob/master/code/Modeling/Stacking_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비

In [1]:
import pandas as pd
import numpy as np

In [2]:
NS = pd.read_csv("NS_2019_m.csv", encoding= 'cp949')
NS.shape

(38300, 34)

In [3]:
# NA 처리(카테고리 : 무형)
NS.isnull().sum()

date             0
product          0
add            932
rain             0
cloud            0
temp             0
ymd              0
wday             0
hour             0
primetime        0
FlowSubway       0
FlowCar          0
duration         0
m_code           0
p_code           0
category         0
price            0
total          932
Y              932
sale_m         932
hms              0
month            0
day              0
holiday          0
org_time         0
stop             0
order            0
max_order        0
before           0
CPI            932
Shop_Online    932
first            0
second           0
third            6
dtype: int64

In [4]:
NS[NS['Y'].isnull()]

,date,product,add,rain,cloud,temp,ymd,wday,hour,primetime,FlowSubway,FlowCar,duration,m_code,p_code,category,price,total,Y,sale_m,hms,month,day,holiday,org_time,stop,order,max_order,before,CPI,Shop_Online,first,second,third
2318,2019-01-25 19:00:00,TCOM_처브치아보험,NaN,1.3,2,0.381818,2019-01-25,6,19,0,891681,528387,60.0,100507.0,201567.0,무형,0.0,NaN,NaN,NaN,19H 0M 0S,1,25,0,60.0,0,1,1,농수축,NaN,NaN,5,0,7.0
2845,2019-01-31 18:00:00,TCOM_처브치아보험,NaN,67.4,3,0.418182,2019-01-31,5,18,0,1132635,555306,60.0,100507.0,201567.0,무형,0.0,NaN,NaN,NaN,18H 0M 0S,1,31,0,60.0,0,1,1,농수축,NaN,NaN,5,0,7.0
3332,2019-02-05 18:00:00,TCOM_처브치아보험,NaN,0.0,2,4.372727,2019-02-05,3,18,0,252326,470461,60.0,100507.0,201567.0,무형,0.0,NaN,NaN,NaN,18H 0M 0S,2,5,1,60.0,0,1,1,이미용,NaN,NaN,5,0,7.0
6570,2019-03-09 00:00:00,온라인투어 동남아 나트랑/빈펄랜드 (190309),NaN,0.0,1,7.027273,2019-03-08,6,24,0,119887,282308,20.0,100324.0,201095.0,무형,0.0,NaN,NaN,NaN,0S,3,8,0,20.0,0,1,3,속옷,NaN,NaN,3,2,4.0
6571,2019-03-09 00:20:00,온라인투어 동남아 나트랑/빈펄랜드 (190309),NaN,0.0,1,7.027273,2019-03-08,6,24,0,119887,282308,20.0,100324.0,201095.0,무형,0.0,NaN,NaN,NaN,20M 0S,3,8,0,20.0,0,2,3,속옷,NaN,NaN,3,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38179,2019-12-30 18:20:00,"삼성화재 천만안심 건강보험(암보험, 1907)",NaN,6.8,3,5.472727,2019-12-30,2,18,1,1105329,545777,60.0,100072.0,200194.0,무형,0.0,NaN,NaN,NaN,18H 20M 0S,12,30,0,60.0,0,1,1,농수축,NaN,NaN,0,7,2.0
38243,2019-12-31 13:20:00,"삼성화재 천만안심 건강보험(암보험, 1907)",NaN,0.0,1,-3.763636,2019-12-31,3,13,0,503568,530322,60.0,100072.0,200194.0,무형,0.0,NaN,NaN,NaN,13H 20M 0S,12,31,0,60.0,0,1,1,주방,NaN,NaN,0,7,2.0
38289,2019-12-31 22:20:00,넥센타이어 렌탈서비스,NaN,0.0,1,-3.763636,2019-12-31,3,22,0,394194,404472,20.0,100543.0,201624.0,무형,0.0,NaN,NaN,NaN,22H 20M 0S,12,31,0,20.0,0,1,3,가전,NaN,NaN,5,4,3.0
38290,2019-12-31 22:40:00,넥센타이어 렌탈서비스,NaN,0.0,1,-3.763636,2019-12-31,3,22,0,394194,404472,20.0,100543.0,201624.0,무형,0.0,NaN,NaN,NaN,22H 40M 0S,12,31,0,20.0,0,2,3,가전,NaN,NaN,5,4,3.0


In [5]:
NS = NS.dropna()
NS.shape

(37362, 34)

In [6]:
# 필요없는 변수 제거 / 더미화
NS2 = NS.drop(['p_code','m_code','date','hms', 'org_time', 'product','sale_m','ymd'],axis=1) 
NS2 = pd.get_dummies(NS2, columns=['month','day','wday','hour','cloud','order','max_order','before','add', 'first', 'second', 'third'])
NS2.head()

,rain,temp,primetime,FlowSubway,FlowCar,duration,category,price,total,Y,holiday,stop,CPI,Shop_Online,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,...,add_청소,add_청소기,add_침구,add_침대,add_카페트,add_커튼,add_케어,add_팬,add_포트,add_해산물,add_헤어,first_0,first_1,first_2,first_3,first_4,first_5,first_6,first_7,first_8,second_0,second_1,second_2,second_3,second_4,second_5,second_6,second_7,second_8,second_9,third_0.0,third_1.0,third_2.0,third_3.0,third_4.0,third_5.0,third_6.0,third_7.0,third_8.0,third_9.0
0,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,2099000.0,2.630326,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,4371000.0,5.477444,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,3262000.0,4.087719,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,6955000.0,8.715539,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0.1,-1.681818,0,115735,152109,20.0,의류,39900.0,6672000.0,8.360902,1,0,104.02,1.0946,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


# 모델링 준비


## Split to train/test

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit


# class 나누기 전 우선 train, test-set으로 split
# empty column 'class_' 추가 (-> target 변수 )
NS2=NS2.assign(class_="")

grp = NS2['category']

In [8]:
# train : test = 0.75 : 0.25 (category stratified)
splitt = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=2020)

In [9]:
for train_index, test_index in splitt.split(NS2, grp):
    strat_train = NS2.iloc[train_index,]
    strat_test = NS2.iloc[test_index,]

In [38]:
strat_train = NS2[(NS2['month_1']==1)|
                  (NS2['month_2']==1)|
                  (NS2['month_3']==1)|
                  (NS2['month_4']==1)|
                  (NS2['month_5']==1)|
                  (NS2['month_6']==1)]

strat_test = NS2[(NS2['month_12'] ==1)|
                 (NS2['month_11'] ==1)]

In [39]:
# 카테고리 더미화 
strat_train = pd.get_dummies(strat_train, columns=['category'])
strat_test = pd.get_dummies(strat_test,  columns=['category'])

## 1. Price

In [40]:
q1=NS2['price'].quantile(q=0.25, interpolation='nearest')
q3=NS2['price'].quantile(q=0.75, interpolation='nearest')

In [41]:
# 위 quantile을 기준으로 train, test set 모두 class 1,2,3으로 분류 
strat_train['class_'].loc[strat_train['price'] <q1]=1
strat_train['class_'].loc[(strat_train['price']>=q1) & (strat_train['price'] <q3)]=2 
strat_train['class_'].loc[strat_train['price']>=q3]=3

strat_test['class_'].loc[strat_test['price'] <q1]=1
strat_test['class_'].loc[(strat_test['price']>=q1) & (strat_test['price'] <q3)]=2 
strat_test['class_'].loc[strat_test['price']>=q3]=3

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [42]:
### Low / Middle / High

In [43]:
low_train = strat_train[strat_train['class_']==1]
low_test = strat_test[strat_test['class_']==1]

X_low_train = low_train[low_train.columns.difference(['class_','total','Y'])]
y_low_train = low_train['Y']
X_low_test = low_test[low_test.columns.difference(['class_','total','Y'])]
y_low_test = low_test['Y']

print(X_low_train.shape, y_low_train.shape)
print(X_low_test.shape, y_low_test.shape)

(3555, 217) (3555,)
(1291, 217) (1291,)


In [44]:
idx1 = y_low_train[np.log(y_low_train) < -100].index
y_low_train1 = np.log(y_low_train.drop(idx1))
X_low_train1 = X_low_train.drop(idx1)

idx2 = y_low_test[np.log(y_low_test) < -100].index
y_low_test1 = np.log(y_low_test.drop(idx2))
X_low_test1 = X_low_test.drop(idx2)

print(X_low_train1.shape, y_low_train1.shape)
print(X_low_test1.shape, y_low_test1.shape)

(3555, 217) (3555,)
(1289, 217) (1289,)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [45]:
middle_train = strat_train[strat_train['class_']==2]


middle_test = strat_test[strat_test['class_']==2]

X_middle_train = middle_train[middle_train.columns.difference(['class_','total','Y'])]
y_middle_train = middle_train['Y']
X_middle_test = middle_test[middle_test.columns.difference(['class_','total','Y'])]
y_middle_test = middle_test['Y']

print(X_middle_train.shape, y_middle_train.shape)
print(X_middle_test.shape, y_middle_test.shape)

(10021, 217) (10021,)
(3515, 217) (3515,)


In [46]:
idx1 = y_middle_train[np.log(y_middle_train) < -100].index
y_middle_train1 = np.log(y_middle_train.drop(idx1))
X_middle_train1 = X_middle_train.drop(idx1)

idx2 = y_middle_test[np.log(y_middle_test) < -100].index
y_middle_test1 = np.log(y_middle_test.drop(idx2))
X_middle_test1 = X_middle_test.drop(idx2)

print(X_middle_train1.shape, y_middle_train1.shape)
print(X_middle_test1.shape, y_middle_test1.shape)

(9934, 217) (9934,)
(3492, 217) (3492,)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [47]:
high_train = strat_train[strat_train['class_']==3]
high_test = strat_test[strat_test['class_']==3]

X_high_train = high_train[high_train.columns.difference(['class_','total','Y'])]
y_high_train = high_train['Y']
X_high_test = high_test[high_test.columns.difference(['class_','total','Y'])]
y_high_test = high_test['Y']

print(X_high_train.shape, y_high_train.shape)
print(X_high_test.shape, y_high_test.shape)

(5351, 217) (5351,)
(954, 217) (954,)


In [48]:
idx1 = y_high_train[np.log(y_high_train) < -100].index
y_high_train1 = np.log(y_high_train.drop(idx1))
X_high_train1 = X_high_train.drop(idx1)

idx2 = y_high_test[np.log(y_high_test) < -100].index
y_high_test1 = np.log(y_high_test.drop(idx2))
X_high_test1 = X_high_test.drop(idx2)

print(X_high_train1.shape, y_high_train1.shape)
print(X_high_test1.shape, y_high_test1.shape)

(4297, 217) (4297,)
(821, 217) (821,)


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# 모델링

## parameter 최적화

In [20]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=c2330ad51587f67b61ce63af49d92a9f95332cc96c383926db13f3f6c94d4eae
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [21]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

from sklearn.model_selection import cross_val_score as cvs, KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, make_scorer, r2_score

import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score


In [22]:
pbounds = {
    'n_estimators': (300, 1000),
    'max_depth': (4,8),
    'subsample': (0.8, 0.9),  # Change for big datasets
    'colsample_bytree': (0.6, 0.8),   # Change for datasets with lots of features
    'gamma' : (10,20),
    'min_child_weight' : (5,10),
    'max_delta_step' : (0.03, 0.05)
    }

In [23]:
def xgboost_hyper_param1(n_estimators, max_depth, 
                        subsample, colsample_bytree, gamma, 
                         min_child_weight, max_delta_step):
    
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    clf = xgb.XGBRegressor(learning_rate=0.1, n_estimators=n_estimators,
                     max_depth=max_depth, subsample= subsample,
                     colsample_bytree=colsample_bytree, gamma = gamma,
                     min_child_weight = min_child_weight, 
                     max_delta_step = max_delta_step, loss = "mape")
    
    
    clf.fit(X_low_train1, y_low_train1)
    
    
    y_pred = clf.predict(X_low_test1)

    mape = mean_absolute_percentage_error(np.exp(y_low_test1), np.exp(y_pred))

    return -mape

optimizer1 = BayesianOptimization(
    f=xgboost_hyper_param1,
    pbounds=pbounds,
    random_state=2019
)

In [24]:
optimizer1.maximize(init_points=3, n_iter=10, acq='ei')

|   iter    |  target   | colsam... |   gamma   | max_de... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
[13:15:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  1        | -84.57    |  0.7807   |  13.93    |  0.04248  |  6.552    |  9.402    |  509.4    |  0.8702   |
[13:15:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  2        | -82.52    |  0.7806   |  18.81    |  0.03811  |  5.81     |  6.335    |  414.0    |  0.8889   |
[13:16:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
|  3        | -94.42    |  0.6297   |  19.85    |  0.03065  |  6.061    |  6.006    |  920.2    |  0.8514   |
[13:16:31] WARNING: /workspace/src/objective/regression_obj.cu:152: r

-------------------------------------------------

In [25]:
params =  optimizer1.max['params']

In [26]:
params

{'colsample_bytree': 0.6454175455289219,
 'gamma': 18.409677803570855,
 'max_delta_step': 0.04303726490888071,
 'max_depth': 4.617692257409294,
 'min_child_weight': 6.443164778967342,
 'n_estimators': 414.8932200208266,
 'subsample': 0.8323985024387449}

low 

lr = 0.06

{'colsample_bytree': 0.7311606460276696,
 'max_depth': 5.373284727735124,
 'n_estimators': 796.6858550926795,
 'subsample': 0.8872870506359695}

 16.358771094818714
30.330715418012666



In [86]:
xgb_clf1 = xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )

xgb_clf2 = xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.06, 
                            max_depth=5,
                            n_estimators=796,
                            subsample= 0.88,
                            loss = "mape"   ,random_state=2020 )


xgb_clf1.fit(X_low_train1, y_low_train1)
xgb_clf2.fit(X_low_train1, y_low_train1)

[14:14:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:14:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.73, gamma=0,
             importance_type='gain', learning_rate=0.06, loss='mape',
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
             n_estimators=796, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=2020, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.88, verbosity=1)

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RepeatedKFold

In [93]:
level0 = list()
level0.append(('XGB1', xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )))

level0.append(('XGB2', xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )))

level0.append(('XGB3', xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )))

level0.append(('XGB4', xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )))

level0.append(('XGB5', xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.06, 
                            max_depth=5,
                            n_estimators=796,
                            subsample= 0.88,
                            loss = "mape"   ,random_state=2020 )))
# define meta learner model
level1 = LinearRegression()
# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

model.fit(X_low_train1,y_low_train1)

[14:19:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:19:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

StackingRegressor(cv=5,
                  estimators=[('XGB1',
                               XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.73, gamma=18,
                                            importance_type='gain',
                                            learning_rate=0.1, loss='mape',
                                            max_delta_step=0.04, max_depth=6,
                                            min_child_weight=6.44, missing=None,
                                            n_estimators=414, n_jobs=1,
                                            nthread=None,
                                            objective='reg:linear',
                                            random_sta...
                                            max_delta_step=0, max_depth=5,
                   

In [94]:
low_pred_train = model.predict(X_low_train1)
low_pred_test = model.predict(X_low_test1)

print(mean_absolute_percentage_error(np.exp(y_low_train1), np.exp(low_pred_train)))
print(mean_absolute_percentage_error(np.exp(y_low_test1), np.exp(low_pred_test)))

13.136098637169837
114.98685191795292


In [87]:
low_pred_train = xgb_clf1.predict(X_low_train1)
low_pred_test = xgb_clf1.predict(X_low_test1)

print(mean_absolute_percentage_error(np.exp(y_low_train1), np.exp(low_pred_train)))
print(mean_absolute_percentage_error(np.exp(y_low_test1), np.exp(low_pred_test)))

low_pred_train2 = xgb_clf2.predict(X_low_train1)
low_pred_test2 = xgb_clf2.predict(X_low_test1)

print(mean_absolute_percentage_error(np.exp(y_low_train1), np.exp(low_pred_train2)))
print(mean_absolute_percentage_error(np.exp(y_low_test1), np.exp(low_pred_test2)))

76.28821262482558
94.91852891087188
13.107401992518186
116.06309167799569


lr = 0.06

{'colsample_bytree': 0.9862708473909292,
 'max_depth': 6.586455338406134,
 'n_estimators': 709.1631951535353,
 'subsample': 0.953981818429076}

 29.908990450346217
43.72936402331216

In [51]:
xgb_clf2 = xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )


xgb_clf2.fit(X_middle_train1, y_middle_train1)

[13:23:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.73, gamma=18,
             importance_type='gain', learning_rate=0.1, loss='mape',
             max_delta_step=0.04, max_depth=6, min_child_weight=6.44,
             missing=None, n_estimators=414, n_jobs=1, nthread=None,
             objective='reg:linear', random_state=2020, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.83, verbosity=1)

In [52]:
middle_pred_train = xgb_clf2.predict(X_middle_train1)
middle_pred_test = xgb_clf2.predict(X_middle_test1)

print(mean_absolute_percentage_error(np.exp(y_middle_train1), np.exp(middle_pred_train)))
print(mean_absolute_percentage_error(np.exp(y_middle_test1), np.exp(middle_pred_test)))

95.99736195607026
94.4151294743057


lr = 0.04

{'colsample_bytree': 0.8042412547305258,
 'max_depth': 5.231524958506885,
 'n_estimators': 1078.913324661913,
 'subsample': 0.9106219081730786}

 32.32104426057097
53.49181413330286

In [53]:
xgb_clf3 = xgb.XGBRegressor(colsample_bytree=0.73,
                            learning_rate=0.1, 
                            max_depth=6,
                            n_estimators=414,
                            subsample= 0.83,
                            min_child_weight = 6.44,
                            max_delta_step = 0.04,
                            gamma = 18,
                            loss = "mape"   ,random_state=2020 )


xgb_clf3.fit(X_high_train1, y_high_train1)

[13:24:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.73, gamma=18,
             importance_type='gain', learning_rate=0.1, loss='mape',
             max_delta_step=0.04, max_depth=6, min_child_weight=6.44,
             missing=None, n_estimators=414, n_jobs=1, nthread=None,
             objective='reg:linear', random_state=2020, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.83, verbosity=1)

In [54]:
high_pred_train = xgb_clf3.predict(X_high_train1)
high_pred_test = xgb_clf3.predict(X_high_test1)

print(mean_absolute_percentage_error(np.exp(y_high_train1), np.exp(high_pred_train)))
print(mean_absolute_percentage_error(np.exp(y_high_test1), np.exp(high_pred_test)))

79.90618333293808
92.53729201809347


In [55]:
from pandas import Series

In [56]:
index1 = y_low_test1.index
y_pred_1 = Series(low_pred_test, index= index1, name = 'Y_hat')

index2 = y_middle_test1.index
y_pred_2 = Series(middle_pred_test, index= index2, name = 'Y_hat')

index3 = y_high_test1.index
y_pred_3 = Series(high_pred_test, index= index3, name = 'Y_hat')

In [57]:
Y1 = pd.concat([y_low_test1, y_pred_1], axis =1)
Y2 = pd.concat([y_middle_test1, y_pred_2], axis =1)
Y3 = pd.concat([y_high_test1, y_pred_3], axis =1)

In [58]:
Y_df = pd.concat([Y1,Y2,Y3])

mean_absolute_percentage_error(np.exp(Y_df['Y']), np.exp(Y_df['Y_hat']))

94.25575377316032